In [ ]:
import os
import glob
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import make_grid
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
# -----------------------
# 1. Hyperparameters
# -----------------------
image_size = 64
batch_size = 64
z_dim = 100
num_epochs = 100
lr = 0.0002
beta1 = 0.5

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# 3. Custom Dataset (Direct Path)
# -----------------------
class CelebADirectPathDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img

In [ ]:
# -----------------------
# 4. Data Setup
# -----------------------
dataroot = r"D:\sangita-mam\assginment-13\cyclegan_faces\trainA"
image_paths = sorted(glob.glob(os.path.join(dataroot, "*.jpg")))

transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3),
])

dataset = CelebADirectPathDataset(image_paths, transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [ ]:
# -----------------------
# 5. DCGAN Generator
# -----------------------
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(z_dim, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
# -----------------------
# 6. DCGAN Discriminator
# -----------------------
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input).view(-1)

In [ ]:
# -----------------------
# 7. Initialize
# -----------------------
gen = Generator().to(device)
disc = Discriminator().to(device)

gen.apply(lambda m: nn.init.normal_(m.weight.data, 0.0, 0.02) if isinstance(m, (nn.ConvTranspose2d, nn.Conv2d)) else None)
disc.apply(lambda m: nn.init.normal_(m.weight.data, 0.0, 0.02) if isinstance(m, (nn.ConvTranspose2d, nn.Conv2d)) else None)

In [ ]:
# -----------------------
# 8. Optimizer & Loss
# -----------------------
criterion = nn.BCELoss()
opt_gen = torch.optim.Adam(gen.parameters(), lr=lr, betas=(beta1, 0.999))
opt_disc = torch.optim.Adam(disc.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
fixed_noise = torch.randn(10, z_dim, 1, 1, device=device)  # For 2x5 grid

In [ ]:
def show_generated_images(generator, fixed_noise, epoch):
    generator.eval()
    with torch.no_grad():
        fake_images = generator(fixed_noise).detach().cpu()
    grid = make_grid(fake_images, nrow=5, normalize=True)
    plt.figure(figsize=(10, 4))
    plt.imshow(np.transpose(grid, (1, 2, 0)))
    plt.title(f"Generated Faces at Epoch {epoch}")
    plt.axis('off')
    plt.show()
    generator.train()


In [ ]:
# -----------------------
# 10. Training Loop
# -----------------------
for epoch in range(1, num_epochs + 1):
    for real in dataloader:
        real = real.to(device)
        batch_size = real.size(0)

        # === Train Discriminator ===
        z = torch.randn(batch_size, z_dim, 1, 1, device=device)
        fake = gen(z)
        loss_real = criterion(disc(real), torch.ones(batch_size, device=device))
        loss_fake = criterion(disc(fake.detach()), torch.zeros(batch_size, device=device))
        loss_disc = (loss_real + loss_fake) / 2

        opt_disc.zero_grad()
        loss_disc.backward()
        opt_disc.step()

        # === Train Generator ===
        output = disc(fake)
        loss_gen = criterion(output, torch.ones(batch_size, device=device))

        opt_gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()

    print(f"Epoch [{epoch}/{num_epochs}]  Loss D: {loss_disc:.4f}, Loss G: {loss_gen:.4f}")

    # Show generated images every epoch
    show_generated_images(gen, fixed_noise, epoch)